In [0]:
from modely import *
from keras.models import load_model
import random
size = 512
#model = load_model("human_weights-004-0.1119.hdf5", custom_objects={"awesomeq_loss":awesomeq_loss})#unet(None,input_size = (size,size,3))
import os
import cv2
fname = "human_weights-{epoch:03d}-{val_loss:.4f}.hdf5"
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(fname, monitor="val_loss",
save_best_only=True, verbose=1)
callbacks = [checkpoint]
from keras.preprocessing.image import ImageDataGenerator

batch = 16


def brightness_adjustment(img):
    #print(img.shape)
    if(img.shape[2] == 3):
      # turn the image into the HSV space
      hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
      # creates a random bright
      ratio = .5 + np.random.uniform(-0.499,0.799)
      # convert to int32, so you don't get uint8 overflow
      # multiply the HSV Value channel by the ratio
      # clips the result between 0 and 255
      # convert again to uint8
      hsv[:,:,2] =  np.clip(hsv[:,:,2].astype(np.int32) * ratio, 0, 255).astype(np.uint8)
      # return the image int the BGR color space
      return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB).astype(np.uint8)
    
    return img
  
def img_compression(img):
    if img.shape[2] == 3:
    
      encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 50 + np.random.uniform(-40,20)]
      result, encimg = cv2.imencode('.jpg', img, encode_param)
      img = cv2.imdecode(encimg, 1)
      
      return brightness_adjustment( img).astype(np.float64)
    return img

data_gen_args = dict(preprocessing_function= img_compression,rescale = 1/255.0)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)
seed= 1

image_generator = image_datagen.flow_from_directory(
    'train',
    class_mode=None,
    target_size = (size,size),
    classes = ['data'],
    color_mode ='rgb',
    batch_size = batch,
    shuffle=True,
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    'train',
    class_mode=None,
    classes = ['label_1'],
    target_size = (size,size),
    color_mode = 'grayscale',
    batch_size = batch,
    
    seed=seed)

train_generator = zip(image_generator, mask_generator)

vimage_generator = image_datagen.flow_from_directory(
    'val',
    class_mode=None,
    classes = ['data'],
    target_size = (size,size),
    color_mode ='rgb',
    batch_size = batch,
    shuffle = True,
    seed=seed)

vmask_generator = mask_datagen.flow_from_directory(
    'val',
    class_mode=None,
    classes = ['label_1'],
    target_size = (size,size),
    color_mode = 'grayscale',
    batch_size = batch, 
    seed=seed)
val_generator = zip(vimage_generator, vmask_generator)

from keras.optimizers import SGD,Adam, RMSprop

optimizer =  RMSprop(5e-5)
model.compile(optimizer = Adam(lr = 5e-5), loss = awesomeq_loss, metrics = ['accuracy'])


H = model.fit_generator(
        train_generator,
	 callbacks = callbacks,
        steps_per_epoch=6985//batch,
        epochs=20,
        verbose = 1,
        validation_data=val_generator,
        validation_steps=500//batch)

import numpy as np 
import matplotlib.pyplot as plt 

plt.style.use("ggplot")
fig = plt.figure()


plt.plot(np.arange(0, 20), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 20), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 20), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 20), H.history["val_acc"], label="val_acc")
title = "0.01"
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch#")

plt.ylabel("Loss/Accuracy")
plt.legend()
fig.savefig('image.png', dpi=fig.dpi)
plt.show()

Found 6985 images belonging to 1 classes.
Found 6985 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Epoch 1/20
436/436 [==============================] - 462s 1s/step - loss: 0.1158 - acc: 0.9589 - val_loss: 0.1142 - val_acc: 0.9594

Epoch 00001: val_loss improved from inf to 0.11419, saving model to human_weights-001-0.1142.hdf5
Epoch 2/20
436/436 [==============================] - 384s 881ms/step - loss: 0.1096 - acc: 0.9606 - val_loss: 0.1123 - val_acc: 0.9600

Epoch 00002: val_loss improved from 0.11419 to 0.11235, saving model to human_weights-002-0.1123.hdf5
Epoch 3/20
 17/436 [>.............................] - ETA: 5:55 - loss: 0.1016 - acc: 0.9623

In [0]:
|

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp /content/gdrive/My\ Drive/data1.zip /content/

In [0]:
!unzip data1.zip

Archive:  data1.zip
   creating: val/
   creating: val/data/
  inflating: val/data/1005168618.png  
  inflating: val/data/1026080744.png  
  inflating: val/data/1037510645.png  
  inflating: val/data/1125935090.png  
  inflating: val/data/1152704323.png  
  inflating: val/data/1192669923.png  
  inflating: val/data/1197260798.png  
  inflating: val/data/1218173313.png  
  inflating: val/data/1225031883.png  
  inflating: val/data/1284115562.png  
  inflating: val/data/1302520246.png  
  inflating: val/data/1368322482.png  
  inflating: val/data/1390258080.png  
  inflating: val/data/1421286812.png  
  inflating: val/data/1469022716.png  
  inflating: val/data/1476264906.png  
  inflating: val/data/1477659049.png  
  inflating: val/data/148061624.png  
  inflating: val/data/1499834535.png  
  inflating: val/data/1509221770.png  
  inflating: val/data/1521322299.png  
  inflating: val/data/1529112438.png  
  inflating: val/data/1544442978.png  
  inflating: val/data/1546912464.png  
  in

In [0]:
import os 

import numpy as np
import cv2
img_paths = os.listdir("train/data/")
empty = []
for path in img_paths:
  if path.endswith(".png"):
    empty.append(cv2.cvtColor(cv2.imread("train/data/"+path), cv2.COLOR_BGR2RGB))

In [0]:
import numpy as np
empty = np.array(empty)

NameError: ignored

In [0]:
empty.shape

NameError: ignored

In [0]:
!rm -rf human*

In [0]:
!wget a.zip https://storage.googleapis.com/kaggle-competitions-data/kaggle/10700/371136/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1557161552&Signature=a9ZbFBtCmhXONVwtRI56V1ZHk%2B%2FDUX%2FU2Vc3N9cpsSO%2BQUu9ZboqOAs8bw0jk9QQoqSsut0%2FGJVSCwQDsFHRVwwb2Khl6a5yOonywalmq2BdGKG0RiuzS%2Fx2ZojZZErN4c2nitRU3%2BM1N0Hep5ptSDkhq24LdBMmWNCdslrshAy8C4pLMYKs3825mJzJdQ6oxXgN1opIvG5%2FRMH2DVSPGoBzznfouJBd%2B%2FN98bJR7wL%2BZ%2Ffk9McBBcEEUIBN1ceR%2FTSNisTZ8CwI0CWkYFalJ4Uaa%2BLR30rZwM%2B9MFAoGhX0%2FCUNsizoCOepoLSwI5LrQEQCrnPKICjlfJzDZcr%2FQA%3D%3D&response-content-disposition=attachment%3B+filename%3Dfreesound-audio-tagging-2019.zip

--2019-05-03 16:54:32--  http://a.zip/
Resolving a.zip (a.zip)... /bin/bash: response-content-disposition=attachment%3B+filename%3Dfreesound-audio-tagging-2019.zip: command not found
failed: Name or service not known.
wget: unable to resolve host address ‘a.zip’
--2019-05-03 16:54:33--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/10700/371136/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.201.128, 2607:f8b0:4001:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.201.128|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2019-05-03 16:54:33 ERROR 403: Forbidden.

